In [ ]:
# takes left and right image, put them together and resize them for labelbox.

In [ ]:
import json
import os
import shutil
import glob

In [ ]:
results = json.load(open('/root/data/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/detection_full.json'))

In [ ]:
print("Number of images: {}".format(len(results["images"])))
print("Number of annotations: {}".format(len(results["annotations"])))
print("Number of unique annotations: {}".format(len(list(set([k['image_id'] for k in results["annotations"]])))))

In [ ]:
# create image dic for faster search
image_dic = {}
for img in results["images"]:
    image_dic[img["id"]] = img

In [ ]:
# create ann dic for faster search
ann_dic = {}
for ann in results["annotations"]:
    ann_dic[ann["image_id"]] = ann

In [ ]:
# list all the images paths with detections
paths = []
for ann in results['annotations']:
    img_path = image_dic[ann['image_id']]["local_path"]
    paths.append((img_path, ann["image_id"]))
print(len(paths))

In [ ]:
# # good pairs manually filtered
# good_ts = []
# for image_path in glob.glob('/root/data/headtail/good_pairs/*'):
#     ts = int(os.path.basename(image_path).split('.')[0].split('_')[-1])
#     good_ts.append(ts)
# good_ts = list(set(good_ts))

In [ ]:
# timestamps dict
timestamps = {}
for path in paths:
    path0 = path[0]
    ts = int(os.path.basename(path0).split('.')[0].split('_')[-1])
#     if ts not in good_ts:
#         continue
    side = os.path.basename(path0).split('.')[0].split('_')[0]
    if ts not in timestamps:
        timestamps[ts] = {}
    timestamps[ts][side] = path0
    timestamps[ts][side + "_id"] = path[1]

In [ ]:
good_pairs = [(k,v) for (k, v) in timestamps.items() if "right" in v and "left" in v]

In [ ]:
print("Number of pairs: {}".format(len(good_pairs)))

#### STACK IMAGES

In [ ]:
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
for gp in good_pairs:
    left = cv2.imread(gp[1]['left'])
    right = cv2.imread(gp[1]['right'])
    conc = np.concatenate([right, left], axis=1)
    conc = cv2.resize(conc, (1024, 375))
    cv2.imwrite('/root/data/tolabelbox/{}.jpg'.format(gp[0]), conc)